In [1]:
!pip install pandas
!pip install requests
!pip install bs4
!pip install plotly
!pip install numpy
!pip install geopy
!pip install html5lib
!pip install sqlalchemy==1.3.9
!pip install ipython-sql
!pip install psycopg2-binary

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import clear_output
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import html5lib
import psycopg2
import json

In [3]:
# load data from json

# with open('Data_to_Otomoto.json') as file:
#     Data_to_Otomoto = json.loads(file.read())

# car_brand = Data_to_Otomoto["car_brand"]

In [4]:
insert_table = [
    'ID',
    'Cena',
    'Oferta_od',
    'Kategoria',
    'Marka_pojazdu',
    'Model_pojazdu',
    'Rodzaj_paliwa',
    'Moc',
    'Pojemnosc_skokowa',
    'Liczba_drzwi',
    'Skrzynia_biegow',
    'Wersja',
    'Generacja',
    'Segment',
    'Kolor',
    'Przebieg',
    'Rok_produkcji',
    'Naped',
    'Spalanie_W_Miescie',
    'Typ_nadwozia',
    'Emisja_CO2',
    'Liczba_miejsc',
    'Rodzaj_koloru',
    'Kraj_pochodzenia',
    'Pierwszy_wlasciciel',
    'Bezwypadkowy',
    'Serwisowany_w_ASO',
    'Stan',
    'VAT_marza',
    'Pierwsza_rejestracja',
    'Numer_rejestracyjny_pojazdu',
    'Zarejestrowany_w_Polsce',
    'Filtr_czastek_stalych',
    'Tuning',
    'Mozliwosc_finansowania',
    'Uszkodzony',
    'Faktura_VAT',
    'Spalanie_W_Cyklu_Mieszanym',
    'Spalanie_Poza_Miastem',
    'Okres_gwarancji_producenta',
    'Leasing',
    'Miesieczna_rata',
    'lub_do_przebieg_km',
    'Gwarancja_dealerska_w_cenie',
    'Homologacja_ciezarowa',
    'Oplata_poczatkowa',
    'Liczba_pozostalych_rat',
    'Wartosc_wykupu',
    'Kierownica_po_prawej_Anglik',
    'Zarejestrowany_jako_zabytek']
    

In [5]:
html_data = requests.get("https://www.otomoto.pl/oferta/dacia-duster-prestige-130km-faktura-vat-fabryczna-gwarancja-salon-pl-u-ID6DQcyU.html#7ce9f71ee3")
soup = BeautifulSoup(html_data.content, 'html5lib')

In [6]:
eur = 4.57
print(eur)


4.57


In [7]:
def get_price(soup):
    Cena = soup.find('span',attrs={"class":"offer-price__number"}).text
    Cena = Cena.replace(" ","").replace(",",".")
    if "PLN" in Cena:
        Cena = Cena.replace("PLN","")
        Cena = float(Cena)
    elif "EUR" in Cena:
        Cena = Cena.replace("EUR","")
        Cena = float(Cena)
        Cena = Cena * eur
    else:
        Cena = np.NaN
    return Cena

In [8]:
print(get_price(soup))

69900.0


In [9]:
def take_id(soup):
    ID = soup.find('div',attrs={"class":"offer-content__metabar"})
    ID = ID.find('div')
    ID = ID.find('span',attrs={"id":"ad_id"}).text
    return ID

In [10]:
def take_details(soup):
    details = {}    
    for info in soup.find_all('li',attrs={"class":"offer-params__item"}):
        key = info.find('span').text.strip().replace(" ","_").replace("(","").replace(")","")
        key = key.replace("ś","s").replace("ż","z").replace("ó","o").replace("ć","c").replace("ę","e")
        key = key.replace("ł","l").replace("ą","a")
        if key in insert_table:
            value = info.find('div').text.strip()
            details[key] = value
    return details

In [11]:
def take_all(soup):
    try:
        dict_all = {}
        dict_all["ID"] = take_id(soup)
        dict_all["Cena"] = get_price(soup)
        details = take_details(soup)
        return dict_all | details
    except:
        pass

In [12]:
def data_clean(details):
    try:
        details['Przebieg'] = details['Przebieg'].replace("km","").replace(" ","")
    except:
        pass
    try:
        details['Moc'] = details['Moc'].replace("KM","").replace(" ","").replace(",",".")
    except:
        pass
    try:
        details['Pojemnosc_skokowa'] = details['Pojemnosc_skokowa'].replace("cm3","").replace(" ","").replace(",",".")
    except:
        pass
    try:
        details['Spalanie_W_Miescie'] = details['Spalanie_W_Miescie'].replace("l/100km","").replace(" ","").replace(",",".")
    except:
        pass
    try:
        details['Spalanie_W_Cyklu_Mieszanym'] = details['Spalanie_W_Cyklu_Mieszanym'].replace("l/100km","").replace(" ","").replace(",",".")
    except:
        pass
    try:
        details['Spalanie_Poza_Miastem'] = details['Spalanie_Poza_Miastem'].replace("l/100km","").replace(" ","").replace(",",".")
    except:
        pass
    return details

In [13]:
car_brand = [
  "Abarth",
  "Alfa Romeo",
  "Aston Martin",
  "Audi",
  "Bentley",
  "BMW"
#   "Bugatti",
#   "Cadillac",
#   "Chevrolet",
#   "Chrysler",
#   "Citroën",
#   "Dacia",
#   "Daewoo",
#   "Daihatsu",
#   "Dodge",
#   "Donkervoort",
#   "DS",
#   "Ferrari",
#   "Fiat",
#   "Fisker",
#   "Ford",
#   "Honda",
#   "Hummer",
#   "Hyundai",
#   "Infiniti",
#   "Iveco",
#   "Jaguar",
#   "Jeep",
#   "Kia",
#   "KTM",
#   "Lada",
#   "Lamborghini",
#   "Lancia",
#   "Land Rover",
#   "Landwind",
#   "Lexus",
#   "Lotus",
#   "Maserati",
#   "Maybach",
#   "Mazda",
#   "McLaren",
#   "Mercedes-Benz",
#   "MG",
#   "Mini",
#   "Mitsubishi",
#   "Morgan",
#   "Nissan",
#   "Opel",
#   "Peugeot",
#   "Porsche",
#   "Renault",
#   "Rolls-Royce",
#   "Rover",
#   "Saab",
#   "Seat",
#   "Skoda",
#   "Smart",
#   "SsangYong",
#   "Subaru",
#   "Suzuki",
#   "Tesla",
#   "Toyota",
#   "Volkswagen",
#   "Volvo"
]

In [14]:
table_cars = {
    'ID': [],
    'Cena': [],
    'Oferta_od': [],
    'Kategoria': [],
    'Marka_pojazdu': [],
    'Model_pojazdu': [],
    'Rodzaj_paliwa': [],
    'Moc': [],
    'Pojemnosc_skokowa': [],
    'Liczba_drzwi': [],
    'Skrzynia_biegow': [],
    'Wersja': [],
    'Generacja': [],
    'Segment': [],
    'Kolor': [],
    'Przebieg': [],
    'Rok_produkcji': [],
    'Naped': [],
    'Spalanie_W_Miescie': [],
    'Typ_nadwozia': [],
    'Emisja_CO2': [],
    'Liczba_miejsc': [],
    'Rodzaj_koloru': [],
    'Kraj_pochodzenia': [],
    'Pierwszy_wlasciciel': [],
    'Bezwypadkowy': [],
    'Serwisowany_w_ASO': [],
    'Stan': [],
    'VAT_marza': [],
    'Pierwsza_rejestracja': [],
    'Numer_rejestracyjny_pojazdu': [],
    'Zarejestrowany_w_Polsce': [],
    'Filtr_czastek_stalych': [],
    'Tuning': [],
    'Mozliwosc_finansowania': [],
    'Uszkodzony': [],
    'Faktura_VAT': [],
    'Spalanie_W_Cyklu_Mieszanym': [],
    'Spalanie_Poza_Miastem': [],
    'Okres_gwarancji_producenta': [],
    'Leasing': [],
    'Miesieczna_rata': [],
    'lub_do_przebieg_km': [],
    'Gwarancja_dealerska_w_cenie': [],
    'Homologacja_ciezarowa': [],
    'Oplata_poczatkowa': [],
    'Liczba_pozostalych_rat': [],
    'Wartosc_wykupu': [],
    'Kierownica_po_prawej_Anglik': [],
    'Zarejestrowany_jako_zabytek': []
    }

In [15]:
table_cars = pd.DataFrame(table_cars)

In [16]:
Link_ERROR = 0
Read_announcements = 0
new_row = {}
all_data = []
for brand in car_brand:
    brand_link = f"https://www.otomoto.pl/osobowe/{brand}/?search%5Border%5D=created_at%3Adesc&page="
    for page_main in range(1,2): 
        try:
            html_data_link = requests.get(brand_link + str(page_main))
            soup_url = BeautifulSoup(html_data_link.content, 'html5lib')
        except Exception:
            print("Link_ERROR")
            continue
        links = []
        for offer in soup_url.find_all('a',attrs={"class":"offer-title__link"}):
            if "https://www.otomoto.pl" in offer['href']:
                links.append(offer['href']) 
        for page in links:
            print(page)
            html_data = requests.get(page)
            soup = BeautifulSoup(html_data.content, 'html5lib')
            dict_all = take_all(soup)
            dict_all = data_clean(dict_all)
            if dict_all:
                all_data.append(dict_all)
            Read_announcements += 1
            clear_output(wait=True)
            print("Read_announcements ", Read_announcements)
table_cars = table_cars.append(all_data, ignore_index=True)
#table_cars = pd.DataFrame.from_dict(all_data, orient='columns')
delete_duplicates = Read_announcements - len(table_cars.index) 
print("Delete duplicates: ", delete_duplicates)
print("Finish")

Read_announcements  192
Delete duplicates:  0
Finish


In [17]:
all_data

[{'ID': '6088515478',
  'Cena': 61900.0,
  'Oferta_od': 'Firmy',
  'Kategoria': 'Osobowe',
  'Marka_pojazdu': 'Abarth',
  'Model_pojazdu': '595',
  'Rok_produkcji': '2017',
  'Przebieg': '30631',
  'Pojemnosc_skokowa': '1368',
  'Rodzaj_paliwa': 'Benzyna',
  'Moc': '145',
  'Skrzynia_biegow': 'Manualna',
  'Naped': 'Na przednie koła',
  'Typ_nadwozia': 'Auta małe',
  'Liczba_drzwi': '3',
  'Liczba_miejsc': '4',
  'Kolor': 'Czerwony',
  'VAT_marza': 'Tak',
  'Mozliwosc_finansowania': 'Tak',
  'Leasing': 'Tak',
  'Bezwypadkowy': 'Tak',
  'Stan': 'Używane'},
 {'ID': '6088033302',
  'Cena': 71900.0,
  'Oferta_od': 'Firmy',
  'Kategoria': 'Osobowe',
  'Marka_pojazdu': 'Abarth',
  'Model_pojazdu': '595',
  'Rok_produkcji': '2018',
  'Przebieg': '25191',
  'Pojemnosc_skokowa': '1368',
  'Rodzaj_paliwa': 'Benzyna',
  'Moc': '145',
  'Skrzynia_biegow': 'Manualna',
  'Naped': 'Na przednie koła',
  'Typ_nadwozia': 'Auta małe',
  'Liczba_drzwi': '3',
  'Liczba_miejsc': '4',
  'Kolor': 'Szary',
  '

In [18]:
# page_link = brand_link + str(page_main)
# print(page_link)

In [19]:
# table_cars = pd.DataFrame.from_dict(all_data, orient='columns')
# table_cars.head()

In [20]:
# #GIT
# Link_ERROR = 0
# Read_announcements = 0
# new_row = {}
# all_data = []
# for page_main in range(1,4):  
#     try:
#         html_data_link = requests.get(f"https://www.otomoto.pl/osobowe/?search%5Border%5D=created_at%3Adesc&page={page_main}")
#         soup_url = BeautifulSoup(html_data_link.content, 'html5lib')
#     except Exception:
#         print("Link_ERROR")
#         continue
#     links = []
#     for offer in soup_url.find_all('a',attrs={"class":"offer-title__link"}):
#         if "https://www.otomoto.pl" in offer['href']:
#             links.append(offer['href']) 
#     for page in links:
#         html_data = requests.get(page)
#         soup = BeautifulSoup(html_data.content, 'html5lib')
#         dict_all = take_all(soup)
#         dict_all = data_clean(dict_all)
#         all_data.append(dict_all)
#         Read_announcements += 1
#         clear_output(wait=True)
#         print("Read_announcements ", Read_announcements)
# table_cars = pd.DataFrame.from_dict(all_data, orient='columns')
# delete_duplicates = Read_announcements - len(table_cars.index) 
# print("Delete duplicates: ", delete_duplicates)
# print("Finish")

In [21]:
table_cars.head()

,ID,Cena,Oferta_od,Kategoria,Marka_pojazdu,Model_pojazdu,Rodzaj_paliwa,Moc,Pojemnosc_skokowa,Liczba_drzwi,...,Leasing,Miesieczna_rata,lub_do_przebieg_km,Gwarancja_dealerska_w_cenie,Homologacja_ciezarowa,Oplata_poczatkowa,Liczba_pozostalych_rat,Wartosc_wykupu,Kierownica_po_prawej_Anglik,Zarejestrowany_jako_zabytek
0,6088515478,61900.0,Firmy,Osobowe,Abarth,595,Benzyna,145,1368,3,...,Tak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6088033302,71900.0,Firmy,Osobowe,Abarth,595,Benzyna,145,1368,3,...,Tak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6088506602,67900.0,Firmy,Osobowe,Abarth,595,Benzyna,145,1368,3,...,Tak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6085005657,93900.0,Firmy,Osobowe,Abarth,595,Benzyna,107,1368,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6088473397,79900.0,Firmy,Osobowe,Abarth,595,Benzyna,145,1368,3,...,Tak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
for col in table_cars.columns:
    print(col)

ID
Cena
Oferta_od
Kategoria
Marka_pojazdu
Model_pojazdu
Rodzaj_paliwa
Moc
Pojemnosc_skokowa
Liczba_drzwi
Skrzynia_biegow
Wersja
Generacja
Segment
Kolor
Przebieg
Rok_produkcji
Naped
Spalanie_W_Miescie
Typ_nadwozia
Emisja_CO2
Liczba_miejsc
Rodzaj_koloru
Kraj_pochodzenia
Pierwszy_wlasciciel
Bezwypadkowy
Serwisowany_w_ASO
Stan
VAT_marza
Pierwsza_rejestracja
Numer_rejestracyjny_pojazdu
Zarejestrowany_w_Polsce
Filtr_czastek_stalych
Tuning
Mozliwosc_finansowania
Uszkodzony
Faktura_VAT
Spalanie_W_Cyklu_Mieszanym
Spalanie_Poza_Miastem
Okres_gwarancji_producenta
Leasing
Miesieczna_rata
lub_do_przebieg_km
Gwarancja_dealerska_w_cenie
Homologacja_ciezarowa
Oplata_poczatkowa
Liczba_pozostalych_rat
Wartosc_wykupu
Kierownica_po_prawej_Anglik
Zarejestrowany_jako_zabytek


In [23]:
for col in insert_table:
    print(col)

ID
Cena
Oferta_od
Kategoria
Marka_pojazdu
Model_pojazdu
Rodzaj_paliwa
Moc
Pojemnosc_skokowa
Liczba_drzwi
Skrzynia_biegow
Wersja
Generacja
Segment
Kolor
Przebieg
Rok_produkcji
Naped
Spalanie_W_Miescie
Typ_nadwozia
Emisja_CO2
Liczba_miejsc
Rodzaj_koloru
Kraj_pochodzenia
Pierwszy_wlasciciel
Bezwypadkowy
Serwisowany_w_ASO
Stan
VAT_marza
Pierwsza_rejestracja
Numer_rejestracyjny_pojazdu
Zarejestrowany_w_Polsce
Filtr_czastek_stalych
Tuning
Mozliwosc_finansowania
Uszkodzony
Faktura_VAT
Spalanie_W_Cyklu_Mieszanym
Spalanie_Poza_Miastem
Okres_gwarancji_producenta
Leasing
Miesieczna_rata
lub_do_przebieg_km
Gwarancja_dealerska_w_cenie
Homologacja_ciezarowa
Oplata_poczatkowa
Liczba_pozostalych_rat
Wartosc_wykupu
Kierownica_po_prawej_Anglik
Zarejestrowany_jako_zabytek


In [24]:
table_name = 'Otomoto'

In [34]:
table_cars.to_csv (table_name +'.csv', index = False, header=True)

In [35]:
psql = psycopg2.connect(host='192.168.10.163', port='5432', database='Otomoto', user='barto', password='biznes')

In [36]:
cur = psql.cursor()

In [37]:
cur.execute('DROP TABLE IF EXISTS '+ table_name)
createTableCommand = 'CREATE TABLE ' + table_name + '''(ID int NOT NULL,
    Cena float,
    Oferta_od VARCHAR,
    Kategoria VARCHAR,
    Marka_pojazdu VARCHAR,
    Model_pojazdu VARCHAR,
    Rodzaj_paliwa VARCHAR,
    Moc float,
    Pojemnosc_skokowa float,
    Liczba_drzwi float,
    Skrzynia_biegow VARCHAR,
    Wersja VARCHAR,
    Generacja VARCHAR,
    Segment VARCHAR,
    Kolor VARCHAR,
    Przebieg float,
    Rok_produkcji float,
    Naped VARCHAR,
    Spalanie_W_Miescie float,
    Typ_nadwozia VARCHAR,
    Emisja_CO2 float,
    Liczba_miejsc float,
    Rodzaj_koloru VARCHAR,
    Kraj_pochodzenia VARCHAR,
    Pierwszy_wlasciciel VARCHAR,
    Bezwypadkowy VARCHAR,
    Serwisowany_w_ASO VARCHAR,
    Stan VARCHAR,
    VAT_marza VARCHAR,
    Pierwsza_rejestracja VARCHAR,
    Numer_rejestracyjny_pojazdu VARCHAR,
    Zarejestrowany_w_Polsce VARCHAR,
    Filtr_czastek_stalych VARCHAR,
    Tuning VARCHAR,
    Mozliwosc_finansowania VARCHAR,
    Uszkodzony VARCHAR,
    Faktura_VAT VARCHAR,
    Spalanie_W_Cyklu_Mieszanym float,
    Spalanie_Poza_Miastem float,
    Okres_gwarancji_producenta VARCHAR,
    Leasing VARCHAR,
    Miesieczna_rata VARCHAR,
    lub_do_przebieg_km VARCHAR,
    Gwarancja_dealerska_w_cenie VARCHAR,
    Homologacja_ciezarowa VARCHAR,
    Oplata_poczatkowa VARCHAR,
    Liczba_pozostalych_rat VARCHAR,
    Wartosc_wykupu VARCHAR,
    Kierownica_po_prawej_Anglik VARCHAR,
    Zarejestrowany_jako_zabytek VARCHAR,
    PRIMARY KEY (ID)
    )'''
cur.execute(createTableCommand)
psql.commit()

In [38]:
print(table_cars.columns)

Index(['ID', 'Cena', 'Oferta_od', 'Kategoria', 'Marka_pojazdu',
       'Model_pojazdu', 'Rodzaj_paliwa', 'Moc', 'Pojemnosc_skokowa',
       'Liczba_drzwi', 'Skrzynia_biegow', 'Wersja', 'Generacja', 'Segment',
       'Kolor', 'Przebieg', 'Rok_produkcji', 'Naped', 'Spalanie_W_Miescie',
       'Typ_nadwozia', 'Emisja_CO2', 'Liczba_miejsc', 'Rodzaj_koloru',
       'Kraj_pochodzenia', 'Pierwszy_wlasciciel', 'Bezwypadkowy',
       'Serwisowany_w_ASO', 'Stan', 'VAT_marza', 'Pierwsza_rejestracja',
       'Numer_rejestracyjny_pojazdu', 'Zarejestrowany_w_Polsce',
       'Filtr_czastek_stalych', 'Tuning', 'Mozliwosc_finansowania',
       'Uszkodzony', 'Faktura_VAT', 'Spalanie_W_Cyklu_Mieszanym',
       'Spalanie_Poza_Miastem', 'Okres_gwarancji_producenta', 'Leasing',
       'Miesieczna_rata', 'lub_do_przebieg_km', 'Gwarancja_dealerska_w_cenie',
       'Homologacja_ciezarowa', 'Oplata_poczatkowa', 'Liczba_pozostalych_rat',
       'Wartosc_wykupu', 'Kierownica_po_prawej_Anglik',
       'Zarejestrow

In [39]:
a = 0
for row in table_cars.itertuples():
    full=""
    full_fit="INSERT INTO " + table_name + "("
    for (column, insert) in zip(row, insert_table):
        if str(column) =='nan':
            continue
        column = str(column)
        column = column.replace("'", "''")
        full +="'"+column+"'"+", "
        full_fit += insert + ', '  
        
    full=full[:-2]
    full_fit=full_fit[:-2]
    full_fit=full_fit+')'
    Value_data='VALUES ('+full+');'
    insertDataCommand = full_fit+"\r\n"+Value_data
    cur.execute(insertDataCommand)
psql.commit()
print("Finish")

InvalidTextRepresentation: invalid input syntax for type double precision: "Benzyna"
LINE 2: ..., '61900.0', 'Firmy', 'Osobowe', 'Abarth', '595', 'Benzyna',...
                                                             ^


In [ ]:
row(ID)